In [ ]:
# Import
import os
import pandas as pd
import numpy as np

from pathlib import Path

from CellPacking.tissuegeneration import sheet_init, symetric_circular
from CellPacking.dynamics import (Compression, 
                                  AnisotropicLineTension, 
                                  ShearPlanarGeometry, 
                                  PlaneBarrierElasticity)

from tyssue import Sheet
from tyssue import PlanarGeometry
from tyssue.solvers import QSSolver
from tyssue.solvers.viscous import EulerSolver
from tyssue.behaviors.event_manager import EventManager
from tyssue.behaviors.sheet.basic_events import reconnect, reconnect_3D
from tyssue.dynamics import model_factory, effectors
from tyssue.core.history import HistoryHdf5 


import matplotlib.pyplot as plt
from tyssue.draw import sheet_view
from CellPacking.plot import superimpose_sheet_view
from CellPacking.plot import sheet_view as ply_sheet_view

In [ ]:
SIM_DIR = Path('/mnt/sda1/Sophie/Simulations/20220905_w3D')
sim_save_dir = SIM_DIR
try:
    os.mkdir(sim_save_dir)
except FileExistsError:
    pass

In [ ]:
# %pdb

# Apical sheet init

In [ ]:
# Number of cells in x and y axis
nx = 20
ny = 20
noise = 0.3
gamma_0 = 0.2
phi = np.pi/2

apical_sheet, border = symetric_circular(10, gamma_0, phi, 0,  noise=noise)
apical_sheet.face_df['prefered_perimeter'] = 3 * np.sqrt(apical_sheet.face_df['prefered_area'])


In [ ]:
fig, ax = sheet_view(apical_sheet, **{'edge':
                                    {'color':'black',
                                    'colormap':'bwr'}})
ax.set_aspect('equal')
fig.set_size_inches((12, 12))

# QSSolver, Manager & Model definition

We use the QS solver once to start the simulation next with sheet at an equilibrium stage. 

At this stage we don't apply any anisotropic line tension.

In [ ]:
# Solver
solver_qs = QSSolver(with_t1=False, with_t3=False, with_collisions=False)

manager = EventManager()
manager.append(reconnect)
# Model
model = model_factory(
    [
        effectors.FaceAreaElasticity,
        effectors.PerimeterElasticity, 
    ])

In [ ]:
for i in range(10):
    manager.execute(apical_sheet)
    res = solver_qs.find_energy_min(apical_sheet, PlanarGeometry, model, periodic=False, options={"gtol": 1e-8})
    if res.success is False:
        print (i, res.success)
    apical_sheet.vert_df[["x", "y"]] += np.random.normal(scale=1e-3, size=(apical_sheet.Nv, 2))
    PlanarGeometry.update_all(apical_sheet)
    manager.update()

In [ ]:
fig, ax = sheet_view(apical_sheet, **{'edge':
                                    {'color':"black",}})
ax.set_aspect('equal')
fig.set_size_inches((10, 10))

# Monolayer creation

In [ ]:
from tyssue.generation import extrude 
from tyssue import Monolayer
from CellPacking.dynamics import ShearMonolayerGeometry

In [ ]:
apical_sheet.face_df['z'] = 1
apical_sheet.edge_df['z'] = 1
apical_sheet.vert_df['z'] = 1

extruded = extrude(apical_sheet.datasets, method='translation', vector=[0, 0, -2])
monolayer = Monolayer('mono', extruded)

monolayer.sanitize(trim_borders=True, order_edges=True)
monolayer.validate()


monolayer.face_df['prefered_area'] = monolayer.face_df.loc[0,'prefered_area']
monolayer.face_df['prefered_perimeter'] = 3*np.sqrt(monolayer.face_df['prefered_area'])
monolayer.face_df['area_elasticity'] = 1
monolayer.face_df['perimeter_elasticity'] = 0.1
# monolayer.face_df.loc[monolayer.face_df['segment']=='lateral'.index,'perimeter_elasticity'] = 0.1


monolayer.edge_df['gamma_0'] = pd.to_numeric(monolayer.edge_df['gamma_0'])

monolayer.vert_df['barrier_elasticity'] = 280
monolayer.vert_df['z_barrier'] = 0.6

monolayer.update_specs({"settings":{"dt":0.01,
                                     'threshold_length': 0.1,
                                     'p_4': 1,
                                     'p_5p': 1,
                                     'nrj_norm_factor': 1.0, 
                                     'multiplier':3, },
                      "cell": {
                                "x": 0.0,
                                "y": 0.0,
                                "z": 0.0,
                                "is_alive": True,
                                "prefered_volume": 0.7,
                                "volume": 0.7,
                                "volume_elasticity": 0.5,
                                "z_barrier":1.1,
                                },
                      "edge": {
                                "dx": 0.0,
                                "srce": 0,
                                "face": 0,
                                "dy": 0.0,
                                "ny": 0.0,
                                "nx": 0.0,
                                "length": 0.0,
                                "nz": 0.0,
                                "cell": 0,
                                "sub_volume": 0.0,
                                "dz": 0.0,
                                "sub_area": 0.0,
                                "trgt": 0},
                      "vert": {
                                "x": 0.0,
                                "is_active": True,
                                "z": 0.0,
                                "y": 0.0},
                      "face": {
                                "x": 0.0,
                                "is_alive": True,
                                "z": 0.0,
                                "y": 0.0,}
                    })


ShearMonolayerGeometry.update_all(monolayer)

monolayer.face_df['prefered_area'] = monolayer.face_df['area']
monolayer.face_df['prefered_perimeter'] = 2*np.sqrt(monolayer.face_df['prefered_area'])

ShearMonolayerGeometry.update_all(monolayer)

In [ ]:
monolayer_init = monolayer.copy(deep_copy=True)

In [ ]:
fig = superimpose_sheet_view(monolayer.get_sub_sheet('apical'), monolayer.get_sub_sheet('basal'), ['apical', 'basal'])
fig.show()

# Euler solver

In [ ]:
# Manager
manager = EventManager('face')#, track_event=False)

monolayer = monolayer_init.copy(deep_copy=True)
monolayer.get_opposite_faces()

monolayer.get_opposite_faces()
edge_opp_face = monolayer.upcast_face(monolayer.face_df['opposite'])
monolayer.edge_df['is_border']=False
monolayer.edge_df.loc[edge_opp_face[edge_opp_face==-1].index, 'is_border']=True

monolayer.edge_df['opposite'] = pd.to_numeric(monolayer.edge_df['opposite'])
monolayer.edge_df['z'] = pd.to_numeric(monolayer.edge_df['z'])

ShearMonolayerGeometry.update_all(monolayer)

from tyssue.io.meshes import save_triangular_mesh
save_triangular_mesh('monolayer_init.vtk', monolayer)

In [ ]:
# Model
model = model_factory(
    [
        PlaneBarrierElasticity,
#         effectors.LineTension,
        effectors.FaceAreaElasticity,
        effectors.PerimeterElasticity,
        effectors.CellVolumeElasticity,
    ],
)
manager.append(reconnect_3D)
solver_qs = QSSolver(with_t1=False, with_t3=False, with_collisions=False)

for i in range(10):
    print('------------TEMPS------------')
    print(i)
    manager.execute(monolayer)
    
    res = solver_qs.find_energy_min(monolayer, ShearMonolayerGeometry, model, periodic=False, options={"gtol": 1e-8})
    if res.success is False:
        
        print (i, res.success)
    monolayer.vert_df[["x", "y"]] += np.random.normal(scale=1e-3, size=(monolayer.Nv, 2))
    ShearMonolayerGeometry.update_all(monolayer)
    manager.update()
save_triangular_mesh('monolayer_before_forces.vtk', monolayer)

In [ ]:
fig = superimpose_sheet_view(monolayer.get_sub_sheet('apical'), monolayer.get_sub_sheet('basal'), ['apical', 'basal'])
fig.show()

In [ ]:
# Model
from tyssue.io.hdf5 import save_datasets
model = model_factory(
    [
        PlaneBarrierElasticity,
        effectors.LineTension,
        effectors.FaceAreaElasticity,
        effectors.PerimeterElasticity,
        effectors.CellVolumeElasticity,
    ],
)

solver_qs = QSSolver(with_t1=False, with_t3=False, with_collisions=False)

manager.append(reconnect_3D)
for i in range(50):
    print('------------TEMPS------------')
    print(i)
    manager.execute(monolayer)
    
    res = solver_qs.find_energy_min(monolayer, ShearMonolayerGeometry, model, periodic=False, options={"gtol": 1e-8})
    if res.success is False:
        print (i, res.success)
        
    monolayer.vert_df[["x", "y"]] += np.random.normal(scale=1e-3, size=(monolayer.Nv, 2))
    manager.update()
    save_datasets(os.path.join(sim_save_dir,'monolayer'+str(i)+'.hf5'), monolayer)
    
# for j in range(10):
#     res = solver_qs.find_energy_min(monolayer, ShearMonolayerGeometry, model, periodic=False, options={"gtol": 1e-8})
#     save_datasets(os.path.join(sim_save_dir,'monolayer'+str(i+1)+'.hf5'), monolayer)

In [ ]:
fig = superimpose_sheet_view(monolayer.get_sub_sheet('apical'), monolayer.get_sub_sheet('basal'), ['apical', 'basal'])
fig.show()

In [ ]:
connect_to_basal = False
for s in monolayer.vert_df[monolayer.vert_df['segment']=='apical'].index:
    tt = monolayer.edge_df[monolayer.edge_df.srce==s]['trgt']
    for t in tt:
        if monolayer.vert_df.loc[t]['segment']=='basal':
            connect_to_basal=True
            break
        elif monolayer.vert_df.loc[t]['segment']=='lateral':
            connect_to_basal=True
            break
    if connect_to_basal==False:
        print(s, monolayer.edge_df[monolayer.edge_df['srce']==s]['cell'].unique())
    else:
        connect_to_basal=False

        
connect_to_basal = False
for s in monolayer.vert_df[monolayer.vert_df['segment']=='basal'].index:
    tt = monolayer.edge_df[monolayer.edge_df.srce==s]['trgt']
    for t in tt:
        if monolayer.vert_df.loc[t]['segment']=='apical':
            connect_to_basal=True
            break
        elif monolayer.vert_df.loc[t]['segment']=='lateral':
            connect_to_basal=True
            break
    if connect_to_basal==False:
        print(s, monolayer.edge_df[monolayer.edge_df['srce']==s]['cell'].unique())
    else:
        connect_to_basal=False
    

In [ ]:
len(np.unique(monolayer.edge_df[(monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['num_sides']==3].index)) &
                  (monolayer.edge_df['segment']=='lateral')]['cell'])), monolayer.Nc

In [ ]:
from tyssue.io.meshes import save_triangular_mesh
save_triangular_mesh('monolayer.vtk', monolayer)
save_triangular_mesh('monolayer_apical.vtk', monolayer.get_sub_sheet('apical'))
save_triangular_mesh('monolayer_basal.vtk', monolayer.get_sub_sheet('basal'))

In [ ]:
from tyssue.draw.plt_draw import plot_forces, plot_scaled_energies
fig, ax  = plot_scaled_energies(monolayer, ShearMonolayerGeometry, model, scales=[0.1, 2])

fig, ax = sheet_view(monolayer.get_sub_sheet('apical'), **{'edge':
                                    {'color':monolayer.get_sub_sheet('apical').edge_df['gamma'],
                                    'colormap':'bwr'}})
fig, ax = plot_forces(monolayer, ShearMonolayerGeometry, model, list('xy'), 0.1, ax=ax)

fig.set_size_inches((20,20))
fig, ax = plot_forces(monolayer, ShearMonolayerGeometry, model, list('xz'), 0.1 )
fig.set_size_inches((20,20))
fig, ax = plot_forces(monolayer, ShearMonolayerGeometry, model, list('yz'), 0.1 )
fig.set_size_inches((20,20))

In [ ]:
import time
from plyer import notification
notification.notify(
    title = "ALERT!!!",
    message = "It is finish",
    timeout=10
)

In [ ]:
for c in range(monolayer.Nf):
    faces = np.unique(monolayer.edge_df[monolayer.edge_df['cell']==c]['face'])
    for f in faces :
        if monolayer.face_df.loc[f, 'segment']=='apical':
            a_sides = monolayer.face_df.loc[f, "num_sides"]
        elif monolayer.face_df.loc[f, 'segment']=='basal':
            b_sides = monolayer.face_df.loc[f, "num_sides"]
    if a_sides != b_sides : 
        if a_sides-b_sides>1:
            print(c)

In [ ]:
import meshio

def save_color_cell(monolayer, filename, eptm, cell_number, coords=list('xyz'), color=[1., 0., 0., 0.1]):
    
    points, faces = monolayer.vertex_mesh(coords=coords, vertex_normals=False)
    triangles_colours=[]
    cells = []
    cpt=0
    for f in faces:
        if cpt in np.unique(eptm.edge_df[eptm.edge_df['cell']==cell_number]['face']):
            cells.append(("triangle", np.array([f])))
            triangles_colours.append([color])
        cpt+=1

    mesh = meshio.Mesh(points, cells, cell_data={"a":triangles_colours})
    meshio.vtk.write(filename, mesh)
    return points, faces
    

In [ ]:
# from tyssue.io.hdf5 import load_datasets
# i = 2
# monolayer_ = Monolayer("monolayer",load_datasets("monolayer"+str(i)+".hf5"))
# save_triangular_mesh('monolayer_'+str(i)+'.vtk', monolayer_)
# # save_triangular_mesh('monolayer_apical_'+str(i)+'.vtk', monolayer_.get_sub_sheet('apical'))
# # save_triangular_mesh('monolayer_basal_'+str(i)+'.vtk', monolayer_.get_sub_sheet('basal'))
# cells = [198]

# for center_cell in cells:
#     points, faces = save_color_cell(monolayer_, str(center_cell)+'_'+str(i)+'.vtk', monolayer_, center_cell)

In [ ]:
from tyssue.io.meshes import save_triangular_mesh
# save_triangular_mesh('monolayer__.vtk', monolayer_)
cells = [192, 191, 172, 171]
for center_cell in cells:
    print(center_cell)
    points, faces = save_color_cell(monolayer, str(center_cell)+'__.vtk', monolayer, center_cell)


In [ ]:
np.unique(monolayer.edge_df[monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['num_sides']==3].index)]['cell'])

In [ ]:
len(np.unique(monolayer.edge_df[(monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['num_sides']==3].index)) &
                  (monolayer.edge_df['segment']=='lateral')]['cell']))

In [ ]:
monolayer.Nc

In [ ]:
monolayer.edge_df[monolayer.edge_df['face']==954]['cell']

In [ ]:
monolayer.face_df[(monolayer.face_df['segment']=="lateral") & (monolayer.face_df['num_sides']==3)]

In [ ]:
# cell with triangle ab face
np.unique(monolayer.edge_df[
    monolayer.edge_df['srce'].isin(monolayer.vert_df[monolayer.vert_df['segment']=='lateral'].index)]['cell'].to_numpy())

In [ ]:


save_triangular_mesh('monolayer.vtk', monolayer)
cells = [255]
# cells = np.unique(monolayer.edge_df[
#     monolayer.edge_df['srce'].isin(monolayer.vert_df[monolayer.vert_df['segment']=='lateral'].index)]['cell'].to_numpy())

for center_cell in cells:
    # monolayer.edge_df[monolayer.edge_df['srce']==monolayer.vert_df[monolayer.vert_df['segment']=='lateral'].index[0]]['cell'].to_numpy()[0] 
    points, faces = save_color_cell(monolayer, str(center_cell)+'.vtk', monolayer, center_cell)
#     print(center_cell)
#     for c in monolayer.get_neighborhood(center_cell, 1)['cell'].to_numpy():
#         print(c)
#         save_color_cell(monolayer, str(c)+'.vtk', monolayer, c, color=[0., 1., 0., 0.1])

In [ ]:
cells = []

for center_cell in cells:    
    points, faces = save_color_cell(monolayer_init, '_'+str(center_cell)+'.vtk', monolayer_init, center_cell)
    print(center_cell)
    for c in monolayer_init.get_neighborhood(center_cell, 1)['cell'].to_numpy():
        print(c)
        save_color_cell(monolayer_init, '_'+str(c)+'.vtk', monolayer_init, c, color=[0., 1., 0., 0.1])

In [ ]:
plt.plot(np.append(monolayer.edge_df[monolayer.edge_df['face']==70]['sx'].to_numpy(),
                    monolayer.edge_df[monolayer.edge_df['face']==70].iloc[0]['sx']),
         np.append(monolayer.edge_df[monolayer.edge_df['face']==70]['sy'].to_numpy(),
                    monolayer.edge_df[monolayer.edge_df['face']==70].iloc[0]['sy']))


# TEST

In [ ]:
fig, ax = sheet_view(monolayer.get_sub_sheet('apical'), **{'edge':
                                    {'color':'black',
                                    'colormap':'bwr'}})

for id_, val in monolayer.cell_df.iterrows():
    ax.text(val.x, val.y, id_)

ax.set_aspect('equal')
fig.set_size_inches((20, 20))

In [ ]:
lateral_face_index = monolayer.face_df[(monolayer.face_df['segment']=="lateral")&
                                       (monolayer.face_df['num_sides']==4)&
                                      (monolayer.face_df['opposite']!=-1)].index
angle = []
edges = []
for f in lateral_face_index:
    if monolayer.edge_df[(monolayer.edge_df['face']==f)& 
                  (monolayer.edge_df['dz']<0.2) &
                  (monolayer.edge_df['dz']>-0.2)].shape[0]==2:
    
        angle.append(np.arctan2(monolayer.edge_df[(monolayer.edge_df['face']==f) & 
                      (monolayer.edge_df['dz']<0.2) &
                      (monolayer.edge_df['dz']>-0.2)][['dx','dy']].diff(axis=0).iloc[1]['dy'],
              monolayer.edge_df[(monolayer.edge_df['face']==f) & 
                      (monolayer.edge_df['dz']<0.2) &
                      (monolayer.edge_df['dz']>-0.2)][['dx','dy']].diff(axis=0).iloc[1]['dx']))
        edges.append(monolayer.edge_df[(monolayer.edge_df['face']==f) & 
                      (monolayer.edge_df['dz']<0.2) &
                      (monolayer.edge_df['dz']>-0.2)].index[0])
    
angle = np.array(angle)*180/np.pi

angle = [180+a if a<0 else a for a in angle]
angle = [180-a if a>90 else a for a in angle]
angle=np.array(angle)
twist_face = lateral_face_index[np.where(angle>45)]
twist_edges = np.array(edges)[np.where(angle>45)]

twist_face = twist_face[0::2]
twist_edges = twist_edges[0::2]

# cell_ = monolayer.edge_df[(monolayer.edge_df.face.isin(monolayer.face_df.loc[twist_face]['opposite'][0::2].to_numpy()))]['cell']
